In [ ]:
def ITSA(i,InROI, ots, OG_VOL):
    k=1
    ThPrev=0 
    ThRev= ots[i] 
    x=0
    y=0
    
    while ThRev!=ThPrev:
        ThPrev=ThRev
        prefatmask = (InROI[i]>ThRev)
        prefatmask = np.uint8(prefatmask)
        ret, fatmask = cv2.threshold(prefatmask,0,1,cv2.THRESH_BINARY)  #need this??
        if i==0:
            fatcombined_next=OG_VOL[i+1]+fatmask
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
        elif i==(j-1):
            fatcombined_prev=OG_VOL[i-1]+fatmask
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev  
        else:
            fatcombined_prev=OG_VOL[i-1]+fatmask
            fatcombined_next=fatmask+OG_VOL[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)         
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
        im_ff=fatmask.copy()
        h, w = im_ff.shape[:2] #added
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2) 
        #Remove small islands for Non-Z parts
        nonZ =label(im_ff==1)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)
        Z = (im_ff==2)
        prefatseg1=(Z+nonZ_keep) #do we need int cus boolean??
        fatseg1=prefatseg1*InROI[i]
        #Fat and Muscle Quantification
        preMuscSegP=InROI[i]-fatseg1
        MuscSegP=np.ma.masked_where(preMuscSegP == 0, preMuscSegP)
        FatSegP=np.ma.masked_where(fatseg1==0,fatseg1) 
        MuscSegI=np.mean(MuscSegP)
        FatSegI=np.mean(FatSegP)
        ThRev=(1+((FatSegI-MuscSegI)/FatSegI))*MuscSegI
        print (f"Slice #{i+1} Iteration={k}\n\tThPrev={ThPrev}\n\tThRev={ThRev}\n")

        k+=1
        if k==50:
            break
            
    MSKTHRES = preMuscSegP+2*np.std(preMuscSegP)
    fatseg2 = np.where(fatseg1 < MSKTHRES, 0 , fatseg1)    
    coordinates2= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
    im_ff2=np.float32(np.copy(fatseg2))
    h2, w2 = im_ff2.shape[:2] #added
    mask2 = np.zeros((h2+2, w2+2), np.uint8)
    for item in range(len(coordinates2)):
        cv2.floodFill(im_ff2, mask2, (coordinates2[item][1],coordinates2[item][0]), 2) 
    #Remove small islands for Non-Z parts
    nonZ2 =label(im_ff2==1)
    nonZ_keep2 = (morphology.remove_small_objects(nonZ2,min_size=8, connectivity=1))
    ret2, nonZ_keep2= cv2.threshold(np.uint8(nonZ_keep2),0,1,cv2.THRESH_BINARY)
    Z2 = (im_ff2==2)
    prefatseg2=(Z2+nonZ_keep2) #do we need int cus boolean??
    finalfatseg=np.uint8(prefatseg2*InROI[i])
    
    MuscSeg = InROI[i]-fatseg2
    thresholds = ThRev
    
    return MuscSeg,finalfatseg,thresholds